<a href="https://colab.research.google.com/github/nishJay/FocussedRegionsImageFusion/blob/main/GaussianMethodFocussedRegionImageFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!apt-get -qq install -y build-essential
!apt-get -qq install -y libopencv-dev


In [34]:
%%writefile code.cpp

#include <iostream>
#include <opencv2/opencv.hpp>

using namespace std;
using namespace cv;

// Function to create Gaussian pyramid
void createGaussianPyramid(const Mat& inputImage, vector<Mat>& pyramid, int numOctaves) {
    Mat currentImage = inputImage.clone();
    pyramid.push_back(currentImage);

    for (int octave = 1; octave <= numOctaves; octave++) {
        Mat blurredImage;
        GaussianBlur(currentImage, blurredImage, Size(5, 5), 1.6);

        Mat downsampledImage;
        pyrDown(blurredImage, downsampledImage);

        currentImage = downsampledImage;
        pyramid.push_back(currentImage);
    }
}

// Function to create Laplacian pyramid from Gaussian pyramid
void createLaplacianPyramid(const vector<Mat>& gaussianPyramid, vector<Mat>& laplacianPyramid) {
    for (size_t i = 0; i < gaussianPyramid.size() - 1; i++) {
        Mat expandedImage;
        pyrUp(gaussianPyramid[i + 1], expandedImage, gaussianPyramid[i].size());

        Mat laplacian;
        subtract(gaussianPyramid[i], expandedImage, laplacian);
        laplacianPyramid.push_back(laplacian);
    }
    laplacianPyramid.push_back(gaussianPyramid.back());
}

// Function to fuse base layers of the Gaussian pyramids
Mat fuseBaseLayers(const Mat& baseLayer1, const Mat& baseLayer2, const Mat& saliencyMap) {
    Mat fusedBase;
    // Perform weighted average fusion based on saliency map
    addWeighted(baseLayer1, 0.5, baseLayer2, 0.5, 0, fusedBase);
    return fusedBase;
}

// Function to fuse detail layers of the Gaussian pyramids
Mat fuseDetailLayers(const Mat& detailLayer1, const Mat& detailLayer2) {
    Mat fusedDetail;
    // Fuse detail layers by selecting higher gradients or contrast
    Mat gradient1, gradient2;
    Sobel(detailLayer1, gradient1, CV_32F, 1, 1);
    Sobel(detailLayer2, gradient2, CV_32F, 1, 1);

    double norm1 = norm(gradient1);
    double norm2 = norm(gradient2);

    if (norm1 > norm2)
        fusedDetail = detailLayer1.clone();
    else
        fusedDetail = detailLayer2.clone();

    return fusedDetail;
}

// Function to reconstruct fused pyramid
Mat reconstructFusedPyramid(const vector<Mat>& fusedBaseLayers, const vector<Mat>& fusedDetailLayers) {
    Mat currentImage = fusedBaseLayers.back();

    for (int i = fusedBaseLayers.size() - 2; i >= 0; i--) {
        Mat expandedImage;
        pyrUp(currentImage, expandedImage, fusedBaseLayers[i].size());

        Mat fusedImage;
        add(fusedBaseLayers[i], expandedImage, fusedImage);
        currentImage = fusedImage + fusedDetailLayers[i];
    }

    return currentImage;
}

int main() {
    // Load the input images - IR and VIS
    Mat infraredImage = imread("/content/drive/MyDrive/dataset (1)/multi-focus-image-fusion-dataset/source03_1.tif");
    Mat visibleImage = imread("/content/drive/MyDrive/dataset (1)/multi-focus-image-fusion-dataset/source03_2.tif");

    // Perform image fusion using Gaussian pyramid
    int numOctaves = log(min(infraredImage.cols, infraredImage.rows)) / log(2) - 2;

    // Create Gaussian pyramids for both images
    vector<Mat> infraredPyramid, visiblePyramid;
    createGaussianPyramid(infraredImage, infraredPyramid, numOctaves);
    createGaussianPyramid(visibleImage, visiblePyramid, numOctaves);

    // Create Laplacian pyramids from the Gaussian pyramids
    vector<Mat> infraredLaplacian, visibleLaplacian;
    createLaplacianPyramid(infraredPyramid, infraredLaplacian);
    createLaplacianPyramid(visiblePyramid, visibleLaplacian);

    // Fuse base layers of the Gaussian pyramids
    vector<Mat> fusedBaseLayers;
    for (size_t i = 0; i < infraredLaplacian.size(); i++) {
        Mat fusedBase = fuseBaseLayers(infraredLaplacian[i], visibleLaplacian[i], infraredImage);
        fusedBaseLayers.push_back(fusedBase);
    }

    // Fuse detail layers of the Gaussian pyramids
    vector<Mat> fusedDetailLayers;
    for (size_t i = 0; i < infraredLaplacian.size(); i++) {
        Mat fusedDetail = fuseDetailLayers(infraredLaplacian[i], visibleLaplacian[i]);
        fusedDetailLayers.push_back(fusedDetail);
    }

    // Reconstruct the fused pyramid
    Mat fusedImage = reconstructFusedPyramid(fusedBaseLayers, fusedDetailLayers);

    // Display the final fused image
    // Save the processed image
    cv::imwrite("/content/image.jpg", fusedImage);

    waitKey(0);

    return 0;
}




Overwriting code.cpp


In [35]:
!g++  /content/code.cpp -o /content/code `pkg-config --cflags --libs opencv4`


In [36]:
!/content/code
